# Computes the FF either analytically or filtering computationally and calculating SNR / SNR_res
### Note that for high chi, increasing the srate may be necessary

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import numpy as np
import matplotlib.pyplot as pl
import qnm_filter
import qnm
import astropy.constants as c
T_MSUN = c.M_sun.value * c.G.value / c.c.value**3

In [3]:
mode_inj = '220'
mode_filt = '440'
phi = 0 #np.random.uniform(0, 2*np.pi)

In [4]:
# Maximum likelihood from the cluster
idx0 = 143058
idx1 = 195420
delta_mass = 0.1
delta_chi = 0.005
massspace = np.arange(1, 100, delta_mass) #[1,160]
chispace = np.arange(0.0, 0.99, delta_chi)
param_space = np.array([(i, j) for i in massspace for j in chispace])
mass_grid, chi_grid = np.meshgrid(massspace, chispace)
print("Injected ML: mass = %.2f, chi=%.2f" % (mass_grid.flatten()[idx0], chi_grid.flatten()[idx0]))
print("Filtered ML: mass = %.2f, chi=%.2f" % (mass_grid.flatten()[idx1], chi_grid.flatten()[idx1]))

theoretical = [50, 0.7]
injected = [mass_grid.flatten()[idx0], chi_grid.flatten()[idx0]]
filtered = [mass_grid.flatten()[idx1], chi_grid.flatten()[idx1]]

Injected ML: mass = 50.80, chi=0.72
Filtered ML: mass = 40.00, chi=0.98


In [5]:
w_inj, _, _ = qnm.modes_cache(s=-2,l=int(mode_inj[0]),m=int(mode_inj[1]),n=int(mode_inj[2]))(a=theoretical[1])
w_filt, _, _ = qnm.modes_cache(s=-2,l=int(mode_filt[0]),m=int(mode_filt[1]),n=int(mode_filt[2]))(filtered[1])
w_inj /= theoretical[0]
w_filt /= filtered[0]

filt = (w_inj-w_filt)/(w_inj-np.conj(w_filt))*(w_inj+np.conj(w_filt))/(w_inj+w_filt)
B = np.abs(filt)
psi = -np.angle(filt) #The (-) sign is negative and I think because of different conventions
f_inj = np.real(w_inj)/(2*np.pi)
tau_inj = -1/np.imag(w_inj)
wt = 2*np.pi*f_inj*tau_inj

In [6]:
temp0 = B**2*(1+wt**2+np.cos(2*(phi+psi))-wt*np.sin(2*(phi+psi)))
temp1 = 1+wt**2+np.cos(2*phi) - wt*np.sin(2*phi)
analytical_FF = np.sqrt(1-temp0/temp1)
print(analytical_FF)
print(0.5*(1-analytical_FF**2))

0.11508918252865771
0.49337724003244265


In [10]:
theoretical = [70, 0.7]
filtered = [70, 0.7]

w_inj, _, _ = qnm.modes_cache(s=-2,l=int(mode_inj[0]),m=int(mode_inj[1]),n=int(mode_inj[2]))(a=theoretical[1])
w_filt, _, _ = qnm.modes_cache(s=-2,l=int(mode_filt[0]),m=int(mode_filt[1]),n=int(mode_filt[2]))(filtered[1])
w_inj /= theoretical[0]
w_filt /= filtered[0]

filt = (w_inj-w_filt)/(w_inj-np.conj(w_filt))*(w_inj+np.conj(w_filt))/(w_inj+w_filt)
B = np.abs(filt)
psi = -np.angle(filt) #The (-) sign is negative and I think because of different conventions
f_inj = np.real(w_inj)/(2*np.pi)
tau_inj = -1/np.imag(w_inj)
wt = 2*np.pi*f_inj*tau_inj

temp0 = B**2*(1+wt**2+np.cos(2*(phi+psi))-wt*np.sin(2*(phi+psi)))
temp1 = 1+wt**2+np.cos(2*phi) - wt*np.sin(2*phi)
analytical_FF = np.sqrt(1-temp0/temp1)
print(analytical_FF)
print(0.5*(1-analytical_FF**2))

0.35043161180959226
0.4385988427222656


In [10]:
def compute_FF(mass, chi):
    w_inj, _, _ = qnm.modes_cache(s=-2,l=int(mode_inj[0]),m=int(mode_inj[1]),n=int(mode_inj[2]))(a=theoretical[1])
    w_filt, _, _ = qnm.modes_cache(s=-2,l=int(mode_filt[0]),m=int(mode_filt[1]),n=int(mode_filt[2]))(chi)
    w_inj /= theoretical[0]
    w_filt /= mass
    
    filt = (w_inj-w_filt)/(w_inj-np.conj(w_filt))*(w_inj+np.conj(w_filt))/(w_inj+w_filt)
    B = np.abs(filt)
    psi = -np.angle(filt) #The (-) sign is negative and I think because of different conventions
    f_inj = np.real(w_inj)/(2*np.pi)
    tau_inj = -1/np.imag(w_inj)
    wt = 2*np.pi*f_inj*tau_inj

    temp0 = B**2*(1+wt**2+np.cos(2*(phi+psi))-wt*np.sin(2*(phi+psi)))
    temp1 = 1+wt**2+np.cos(2*phi) - wt*np.sin(2*phi)
    analytical_FF = np.sqrt(1-temp0/temp1)

    return(analytical_FF, 0.5*(1-analytical_FF**2))

In [19]:
mode_inj = '220'
mode_filt = '200'

In [22]:
delta_mass = 0.1
delta_chi = 0.005
massspace = np.arange(1, 160, delta_mass) #[1,160]
chispace = np.arange(0.0, 0.99, delta_chi)
param_space = np.array([(i, j) for i in massspace for j in chispace])

In [23]:
# Finding the minimum FF with the analytic expression

temp_FF = 0
temp_results = ()
for mass, chi in param_space:
    FF, quadratic = compute_FF(mass, chi)
    if FF > temp_FF:
        temp_FF = FF
        temp_results = (FF, quadratic, mass, chi)

print(temp_results)

(0.9980237164387613, 0.0019743307128815, 40.000000000000036, 0.985)
